In [1]:
import numpy as np
import os, sys
import matplotlib.pyplot as plt
import laspy as lp
import pdal
import pandas as pd
import open3d as o3d
from dbfread import DBF

import itertools

from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
from sklearn.neighbors import NearestNeighbors


# basedir = os.path.dirname(os.getcwd())
basedir = os.path.abspath(os.path.join(os.getcwd() ,"../"))
_py = os.path.join(basedir, 'py')
_data = os.path.join(basedir, 'data')

sys.path.insert(1, _py)
import loads
import lia
import ray as rayt
import lad
import figures

import warnings
warnings.filterwarnings("ignore")

%load_ext autoreload
%autoreload 2

%matplotlib qt
# %matplotlib inline

In [2]:
name = 'kiwifruit_interpine'

In [3]:
# load files
las = loads.loadlaz(name)

# See LAS columns names
point_format = las.point_format
if True:
    print(list(point_format.dimension_names))

['X', 'Y', 'Z', 'intensity', 'return_number', 'number_of_returns', 'synthetic', 'key_point', 'withheld', 'overlap', 'scanner_channel', 'scan_direction_flag', 'edge_of_flight_line', 'classification', 'user_data', 'scan_angle', 'point_source_id', 'gps_time', 'red', 'green', 'blue']


In [4]:
# las to data frame
columns = ['x', 'y', 'z', 'intensity', 'return_number', 'number_of_returns', 'gps_time', 'red', 'green', 'blue']
las = pd.DataFrame(np.vstack((las.x, las.y, las.z, las.intensity, las.return_number, las.number_of_returns, las.gps_time, las.red, las.green, las.blue)).transpose(), columns=columns)

In [ ]:
# shift x and y coordinates near zero
for i in ['x', 'y']:
    ismin = np.min(las[i])
    las[i] = las[i] - ismin

In [ ]:
# Define point cloud with RGB colours
points = np.vstack((las.x, las.y, las.z)).transpose()
colors = np.vstack((las.red, las.green, las.blue)).transpose()

In [ ]:
# trajectory file must be called `trajectory.dbf` anc must contain x, y, z and gpstime
# if trajectory not given, bellow we mock this following x, y and gps_time values from point cloud 
try:
    traj = loads.loaddbf(name)
    traj = pd.DataFrame(iter(traj)) # to pandas
except Exception as e:
    print('Error loading trayectory file:\n %s' %(e))
    print('A mock trajectory will be used.')

    xymin = np.amin(points[:,:2], axis=0)
    xymax = np.amax(points[:,:2], axis=0)
    xymock = np.linspace(xymin, xymax, 50)

    zmock = np.full((len(xymock), 1), 40, dtype=int)

    gpsmin = las.gps_time.min()
    gpsmax = las.gps_time.max()
    gpsmock = np.linspace(gpsmin, gpsmax, len(xymock)).reshape(len(xymock), 1)

    traj = np.append(xymock, zmock, axis=1)
    traj = np.append(traj, gpsmock, axis=1)

    traj = pd.DataFrame(traj, columns=['x', 'y', 'z', 'gpstime'])

points_t = np.vstack((traj.x, traj.y, traj.z)).transpose()

Error loading trayectory file:
 could not find file '/Users/omar/projects/planttech/data/kiwifruit_interpine/trajectory.dbf'
A mock trajectory will be used.


Next, we define a region to run our pipeline as we know much of the point cloud is not relevant for the posterior analysis

In [ ]:
def get_PCsample(p):

    PCsample = (p[:,1] > 2.8*p[:,0] - 120) & (p[:,1] < 2.8*p[:,0] -80)
    PCsample &= (p[:,0] < -2.8*p[:,1] + 500) & (p[:,0] > -2.8*p[:,1] + 400)

    return PCsample

In [ ]:
n = 1000
xymin = np.amin(points[:,:2][::n], axis=0)
xymax = np.amax(points[:,:2][::n], axis=0)

plt.scatter(points[:,0][::n], points[:,1][::n], s=0.5, c='gray', alpha=0.5)

x = np.linspace(xymin[0], xymax[0], 3)
y = np.linspace(xymin[1], xymax[1], 3)
plt.plot(x, 2.8*x - 80, ls='--', lw=0.8, c='r')
plt.plot(x, 2.8*x - 120, ls='--', lw=0.8, c='r')
plt.plot(-2.8*y + 400, y, ls='--', lw=0.8, c='r')
plt.plot(-2.8*y + 500, y, ls='--', lw=0.8, c='r')

PCsample = get_PCsample(points[::n])
print(PCsample.sum())
plt.scatter(points[:,0][::n][PCsample], points[:,1][::n][PCsample], c='g', s=0.5)

plt.xlim(0, 175)
plt.ylim(0, 300)

plt.xlabel(r'$x$', size=20)
plt.ylabel(r'$y$', size=20)


2003


Text(0, 0.5, '$y$')

In [10]:
# plot the two point cloudsb
if True:
    pointslist = [points[::1000], points_t]
    colours = [colors[::1000]/2**16, [1, 0, 0]]
    loads.showPCDS(pointslist, colours)

[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


In [11]:
# define and plot a sample of the point cloud

PCsample = get_PCsample(points)

pointslist = [points[PCsample], points_t]
colours = [colors[PCsample]/2**16, [1, 0, 0]]
if True:
    loads.showPCDS(pointslist, colours)

[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


In [317]:
# las_df = pd.DataFrame(np.append(points, las.gps_time.reshape(len(las.gps_time), 1), axis=1), columns=['x', 'y', 'z', 'gps_time'])

In [12]:
# interpolate trajectory with gps time

df = loads.coordsDF(las, traj)

In [13]:
df.head()

,x,y,z,xs,ys,zs
0,57.8851,52.6814,17.054934,0.000090,0.000153,40.0
1,58.1875,52.5695,17.030534,0.000099,0.000169,40.0
2,58.8075,52.3344,17.004434,0.000117,0.000200,40.0
3,59.1026,52.2180,16.970034,0.000126,0.000215,40.0
4,60.9687,51.4525,16.780334,0.000180,0.000307,40.0


In [15]:
# show sample of beams
if False:
    loads.showbeams(df[15000000:15000300])

## Tree and leaves segmentation

### DBSCAN mask

In [ ]:
plt.scatter(-2.8*las['y'][PCsample] + 400, las['z'][PCsample], s=0.5, c='gray', alpha=0.5)
plt.axhline(19.5, c='g')
plt.axhline(21.5, c='g')

x = np.linspace(xymin[0], xymax[0], 3)
y = np.linspace(xymin[1], xymax[1], 3)
# plt.plot(x, 2.8*x - 80, ls='--', lw=0.8, c='r')
# plt.plot(x, 2.8*x - 120, ls='--', lw=0.8, c='r')
# plt.plot(-2.8*y + 400, y, ls='--', lw=0.8, c='r')
# plt.plot(-2.8*y + 500, y, ls='--', lw=0.8, c='r')

plt.xlabel(r'$-2.8y + 300$', size=20)
plt.ylabel(r'$z$', size=20)

In [ ]:
plt.scatter(2.8*las['x'][PCsample] + 300, las['z'][PCsample], s=0.5, c='gray', alpha=0.5)
plt.axhline(19.5, c='g')
plt.axhline(21.5, c='g')

plt.xlabel(r'$-2.8y + 300$', size=20)
plt.ylabel(r'$z$', size=20)

In [15]:

zmin, zmax, zfar = 19.5, 21.5, 23

keep = PCsample & (las.z > 20.8) & (las.z < zfar)
Ntot = keep.sum()

db = DBSCAN(eps=0.25).fit(points[keep])

labels = {}

for i in set(db.labels_):
    mask = db.labels_ == i
    perc = 100*mask.sum()/Ntot
    if perc > 1:
        # print(i, perc)
        labels[i] = mask

N = len(list(labels.keys()))

# plot the two point clouds
coltmp = plt.cm.jet(np.linspace(0,1,N))[:,0:3]

if True:
    pointslist = [points[keep][val] for val in labels.values()]
    colours = [list(i) for i in coltmp]
    loads.showPCDS(pointslist, colours)

[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


In [16]:
above = np.zeros(len(points), dtype=bool)
keep = PCsample & (las.z > 20.8) & (las.z < zfar)

for val in labels.values():

    above[keep] |= val

keep = PCsample & ~above
rej = PCsample & above

if True:
    pointslist = [points[keep], points[rej]]
    colours = [colors[keep]/2**16, [1, 0, 0]]
    loads.showPCDS(pointslist, colours)

[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


## Percentiles masking

In order to get rid of the stems of kiwifruit trees, first, we take a slide parallel to the stems arange. We have a total of two slides that folow equation:

$$y = 0.34x - 0.5 \\
y = 0.34x + 3.3, $$

each one with width of $0.6$.

In [ ]:
keep = PCsample & (las.z < 1.3) & (las.z > 0.4)

plt.figure(figsize=(16, 8))

plt.scatter(np.array(las.x)[keep], np.array(las.y)[keep], c='k', s=0.01)

x = np.linspace(-30, -10, 20)

plt.plot(x, 0.34*x - 0.5, c='r', lw=1, ls='--')
plt.plot(x, 0.34*x + 3.3, c='r', lw=1, ls='--')

for i in [0.3, -0.3]:

    plt.plot(x, 0.34*x - 0.5 + i, c='g', lw=1, ls='-')
    plt.plot(x, 0.34*x + 3.3 + i, c='g', lw=1, ls='-')

plt.xlabel(r'$x$', size=20)
plt.ylabel(r'$y$', size=20)

plt.xlim(-20, -10)

In [ ]:
slide1 = (np.array(las.y) > 0.34*np.array(las.x) - 0.5 - 0.3) & (np.array(las.y) < 0.34*np.array(las.x) - 0.5 + 0.3)
slide2 = (np.array(las.y) > 0.34*np.array(las.x) + 3.3 - 0.3) & (np.array(las.y) < 0.34*np.array(las.x) + 3.3 + 0.3)

below = np.zeros(len(points), dtype=bool)

for i in [slide1, slide2]:

    keep = PCsample & i & (las.z > 1.5) & (las.z < 2.7) & ~above

    plt.figure(figsize=(16, 8))
    plt.scatter(np.array(las.x)[PCsample & ~above & i][::1], np.array(las.z)[PCsample & ~above & i][::1], s=0.04, c='k')
    plt.axhline(1.5, lw=1, c='k')
    plt.axhline(2.7, lw=1, c='k')

    res, bcmask = loads.remove_outliers(np.array(las.x)[keep], np.array(las.z)[keep], nbins=100, bounds=(2, 98.0))
    below[keep] |= ~bcmask

    plt.ylim(-1,4)

    plt.xlabel(r'$x$', size=20)
    plt.ylabel(r'$z$', size=20)



In [ ]:
# sanity check: check that red dots corresponds to the regions we want to mask out
if False:
    keep = PCsample & ~above & ~below
    rej = PCsample & (above | below)
    pointslist = [points[keep], points[rej]]
    colours = [colors[keep]/2**16, [1, 0, 0]]
    loads.showPCDS(pointslist, colours)

In [ ]:
# foliage

foliage = ~above & ~below & (las.z > 1.5) & (las.z < 2.7)
keep = PCsample & foliage
# rej = PCsample & ~foliage
if False:
    pointslist = [points[keep]]
    colours = [colors[keep]/2**16]
    loads.showPCDS(pointslist, colours)

In [ ]:
N = PCsample.sum()
samp = PCsample & foliage

print('total # points in sample: %i' %(N))
print('total # points within foliage: %i, \t %.2f %%' %(samp.sum(), 100*samp.sum()/N))

In [ ]:
def segtree(df, leaves, show=False):

    trees = {}
    centres = []
    # keepS = PCsample[::100]
    # PCsample = (points[:,1] > 0.34*points[:,0] - 3.0) & (points[:,1] < 0.34*points[:,0] + 5)
    # PCsample &= (points[:,0] < -0.35*points[:,1] - 10) & (points[:,0] > -0.35*points[:,1] - 20)
    # bins = np.arange(10, 20, 1)
    squares = list(itertools.product(np.arange(-20, -10, 1), np.arange(-3, 5, 1)))

    if show:
        plt.figure(figsize=(20, 10))
        

    for num, (i,j) in enumerate(squares):
        # print(i,j)
        keep = np.ones(len(df['x']), dtype=bool)
        keep &= (df['y'] > 0.34*df['x'] + j) & (df['y'] < 0.34*df['x'] + (j+1))
        keep &= (df['x'] > -0.35*df['y'] + i) & (df['x'] < -0.35*df['y'] + (i+1))
        # print(np.sum(keep))

        trees['tree_%s' %(str(num))] = keep
            
        if show:
            plt.scatter(df['x'][leaves & keep], df['y'][leaves & keep], s=0.5, label=i)
            p = ((i+0.5) - 0.35 * (j+0.5)) / (1 + (0.35 * 0.34))
            ya = 0.34*(p) + (j+0.5)
            centres.append([num, p, ya])
            # plt.scatter(p,ya, s=20, c='k')

            box = dict(facecolor='green', edgecolor='black', boxstyle='round,pad=0.5', alpha=0.8)
            text = 'tree_%s' %(str(num))

            # x = np.linspace(-18, -12, 5)
            # plt.plot(x, 0.34*x + j + 0.5, c='r', lw=1, ls='--')
            # plt.plot(x, (x - (i + 0.5))/(-0.35), c='r', lw=1, ls='--')

            plt.text(p, ya, text, size=10, bbox=box)

    if show:
        # plt.scatter(df['x'][leaves], df['y'][leaves], s=0.1, c='k')
        plt.xlabel(r'$x$', size=20)
        plt.ylabel(r'$y$', size=20)
        # plt.show()
        
    return trees, centres


In [ ]:
leaves = PCsample & foliage
trees, centres = segtree(df, leaves, show=True)

In [ ]:
# save centre coordinates of patches
resdir = os.path.join(_data, name, 'lia')
if not os.path.exists(resdir):
    os.makedirs(resdir)
    
df_centres = pd.DataFrame(centres, columns=['tree_id', 'x', 'y'])
df_centres.to_csv(os.path.join(resdir, 'centres.csv'), index=False)

In [ ]:
# show the point cloud from leaves of firs tree only
keep = (trees['tree_0']) & (leaves)
# loads.showPCfromDF(df[keep])

if False:
    pointslist = [points[keep]]
    colours = [colors[keep]/2**16]
    loads.showPCDS(pointslist, colours)

## Intensity vs MLS beam trajectory distance

In [ ]:
sensors = np.vstack((df['xs'].values, df['ys'].values, df['zs'].values)).transpose()

# get the distances between the PC and the sensor
ab = sensors - points
dist = np.sqrt(np.sum((ab) ** 2, axis=1))
dist

In [ ]:
keep = PCsample

f = plt.figure(figsize=(10,6))
plt.hist(dist[keep], 50, density=True)
plt.title('Distance between the PC and sensor', size=15)

distbins = {}

for i in [10, 20, 30, 40, 50]:

    distbins['%s-%s.1' %(str(i), str(i))] = np.logical_and(dist[keep] > i, dist[keep] < i+0.1)
    plt.axvline(i, ls='--', color='k')


intensity = np.array(las.intensity[keep])

f = plt.figure(figsize=(10,6))

for key, val in distbins.items():

    print(key, val.sum())

    bins = np.linspace(intensity.min(), intensity.max(), 50)
    plt.hist(intensity[val], bins=bins, histtype='step', lw=2, density=True, label=key)

plt.legend()
plt.title('Intensity', size=15)
    

# `Leaf Inclination Angle` (LIA) estimation

On the contrary to the mock example, here we can not use function `lia.bestfit_pars_la` to get the best-fit parameters...

In [ ]:

def run_lia(trees, voxel_size_w, kd3_sr, max_nn, voxel_size_h=0.1, savefig=True, savefig_extra=False):

    # load bestfit results      
    for num, (key, val) in enumerate(trees.items()):

        # if num == 0: # for debug
        if True:

            print('********* %s *********' %(key))

            keep = (val) & (leaves)
            df_ = df[['x', 'y', 'z']][keep]
            points = loads.DF2array(df_)
            # print(points[:,2].min())

            # voxel_size_w = 0.01
            # kd3_sr = 0.15
            # max_nn = 40

            text = '%s=%.4f \n %s=%.4f \n %s=%.4f ' %(
                                    'voxel_size_w', voxel_size_w,
                                    'kd3_sr', kd3_sr,
                                    'max_nn', max_nn)
            # print(text)

            lia.leaf_angle(points, name, key, voxel_size_w, kd3_sr, max_nn, save=True,
                                        savefig=savefig, text=text, voxel_size_h=voxel_size_h, ismock=False,
                                        ylim=0.03, ylimh=0.45, savefig_extra=savefig_extra)

In [ ]:
run_lia(trees, voxel_size_w=0.01, kd3_sr=0.15, max_nn=300, 
voxel_size_h=0.05, savefig=False, savefig_extra=True)

## Files for Junqi

Following code takes `results_per_height_tree_<ID>.csv` file adds the `LAD` column and break the `values` array to asign a value to a column. Result is stored in Junqi directory with same name.

In [ ]:
import glob

for file in glob.glob(os.path.join(_data, name, 'lia', 'results_per_*.csv')):
    # print(file)
    df_ = pd.read_csv(file)
    outfile = os.path.join(_data, name, 'lia', 'junqi', file.split('/')[-1])
    a = df_['values'].str[1:-1]
    keep = [len(i) > 0 for i in a]
    a = a.str.split(expand=True)
    df_['lad'] = np.arange(0, len(a))
    pd.concat([df_['zmin'][keep], df_['lad'][keep], a[keep]], axis=1).to_csv(outfile, index=False) 


Below code is deprecated now...

In [ ]:
if False:
    
    import glob

    files = glob.glob(os.path.join(_data, name, 'lia', 'angles*.npy'))
    for file in files:
        outfile = os.path.join(_data, name, 'lia', 'junqi', file.split('/')[-1].split('.')[0]+'.csv')
        pd.DataFrame(np.load(file)).to_csv(outfile, index=False)
        
    for num, (key, val) in enumerate(trees.items()):
        if True:
            keep = (val) & (leaves)
            outfile = os.path.join(_data, name, 'lia', 'junqi', 'heighta_%s.csv' %(key))
            df[['z']][keep].to_csv(outfile, index=False)

# `Leaf Area Density` (LAD) estimation

In [ ]:
for key, val in trees.items():

    keep = (val)
    print(key, np.sum(keep))

In [ ]:
downsample = 0.005
voxel_size = 0.15
# to check everything looks fine
show = False
sample = None

In [ ]:

POINTS = loads.DF2array(df[['x', 'y', 'z']])
SENSORS = loads.DF2array(df[['xs', 'ys', 'zs']])

def get_rays(downsample, voxel_size, sample=None, show=False):

    if downsample is not None:

        resdir = os.path.join(_data, name, 'lad_%s' %(str(downsample)))
        if not os.path.exists(resdir):
            os.makedirs(resdir)

        outdir = os.path.join(resdir, 'inds.npy')
        if os.path.exists(outdir):
            print('inds file already exists for donwnsample of %.3f at %s' %(downsample, outdir))

            inds = np.load(outdir)

            points = POINTS[inds]
            sensors = SENSORS[inds]

        else:

            print('inds not been created yet for donwnsample of %.3f' %(downsample))
            idx = np.random.randint(0, len(df), int(len(df) * downsample))
            inds = np.zeros(len(df), dtype=bool)
            inds[idx] = True

            points = POINTS[inds]
            sensors = SENSORS[inds]

            np.save(outdir, inds)

    else:

        resdir = os.path.join(_data, name, 'lad')
        if not os.path.exists(resdir):
            os.makedirs(resdir)

    if sample is not None:

        idx = np.random.randint(0, len(df), int(sample))
        points = POINTS[idx]
        sensors = SENSORS[idx]

    for key, val in trees.items():

        inPR = (val) & (leaves) & (inds)
        pointsPR = POINTS[inPR]
        m3s = rayt.main(points, sensors, pointsPR, voxel_size, resdir, key, show=show)

In [ ]:
def runall(downsample, voxel_size, kbins=None):
    
    if downsample is not None:
        resdir = os.path.join(_data, name, 'lad_%s' %(str(downsample)))
        inds_file = os.path.join(resdir, 'inds.npy')
        inds = np.load(inds_file)
        print('downsample:', downsample)
    else:
        inds = np.ones(len(df), dtype=bool)
        resdir = os.path.join(_data, name, 'lad')

    isfigures = os.path.join(resdir, 'figures')
    if not os.path.exists(isfigures):
        os.makedirs(isfigures)

    print('voxel_size:', voxel_size)

    for key, val in trees.items():

        inPR = (val) & (leaves) & (inds)
        pointsPR = POINTS[inPR]
        sensorsPR = SENSORS[inPR]

        m3att = lad.compute_attributes(pointsPR, resdir, voxel_size, key)

        # _,_,_, m3scount = lad.density_counts(pointsPR, voxel_size)

        # get in down sample boolean array for LPC size
        inds_ = inds[(val) & (leaves)]
        lias, ws = lad.downsample_lia(name, key, inds_)
        voxk = lad.get_voxk(pointsPR, voxel_size)
        bia = lad.get_bia(pointsPR, sensorsPR)
        # meshfile = lad.get_meshfile(name)

        figext = '%s_%s' %(key, str(voxel_size))
        # figext = None
        alphas_k = lad.alpha_k(bia, voxk, lias, ws, resdir, meshfile=None, figext=figext, 
                                klia=False, use_true_lia=False)

        kmax = m3att.shape[2]
        if kbins is None:
            kbins = int(kmax/15)
        print(kbins)

        # Attribute 2 counts per voxel
        # outdir_count = os.path.join(resdir, 'm3count_%s_%s.npy' %(key, str(voxel_size)))
        # m3pcount = np.load(outdir_count)
        # print('******* outdir_count', outdir_count)

        
        # lads_min = lad.get_LADS(m3att, voxel_size, kbins, alphas_k[:,2], 1)
        # lads_max = lad.get_LADS(m3att, voxel_size, kbins, alphas_k[:,4], 1)
        lads_mid = lad.get_LADS(m3att, voxel_size, kbins, alphas_k[:,6], alpha2=1)
        # lads_mid_w = lad.get_LADS(m3att, voxel_size, kbins, alphas_k[:,6], 1, m3scount=m3scount, m3pcount=m3pcount)
        # lads_mid_counts = lad.get_LADS(m3att, voxel_size, kbins, alphas_k[:,6], 1, m3scount=m3scount, usecounts=True, m3pcount=m3pcount)

        lads_0 = lad.get_LADS(m3att, voxel_size, kbins, alphas_k[:,6]*0+1, alpha2=1)
        # lads_mesh = lad.get_LADS_mesh(meshfile, voxel_size, kbins, kmax)

        lads = {'Correction Mean':lads_mid, 'No Correction':lads_0} #, 'Correction counts':lads_mid_counts}
        # lads = {'Truth':lads_mesh, 'Correction Mean':lads_mid, 'No Correction':lads_0}
        clai = lad.get_clai(m3att, alphas_k)
        attributes_file = os.path.join(resdir, 'm3s_%s_%s.npy' %(key, str(voxel_size)))
        if os.path.isfile(attributes_file):
            RT = 'Y'
        else:
            RT = 'N'
            
        text = {'tree':key, 'VS':voxel_size, 'DS':downsample, 'RT':RT, 'CLAI':np.round(clai, 3)}
        txt = []
        for key, val in text.items():
            txt.append('%s=%s \n' %(key, str(val)))
        text = (' ').join(txt)

        savefig = os.path.join(resdir, 'figures','LAD_%s.png' %(figext))
        figures.plot_lads(lads, text, savefig=savefig)

    # return m3pcounts, m3scount



In [ ]:
downsample = 0.01
voxel_size = 0.1
# to check everything looks fine
show = False
sample = None

get_rays(downsample, voxel_size, sample, show)

## Attribute variation

In [ ]:
for DS in [0.005, 0.01, 0.05, 0.1, 0.2]:

    get_rays(downsample=DS, voxel_size=0.1)

In [ ]:
results = []

for downsample in [0.005, 0.01, 0.05, 0.1, 0.2]:
    for voxel_size in [0.1]:

        if downsample is not None:
            resdir = os.path.join(_data, name, 'lad_%s' %(str(downsample)))
            inds_file = os.path.join(resdir, 'inds.npy')
            inds = np.load(inds_file)
            print('downsample:', downsample)
        else:
            inds = np.ones(len(df), dtype=bool)
            resdir = os.path.join(_data, name, 'lad')

        isfigures = os.path.join(resdir, 'figures')
        if not os.path.exists(isfigures):
            os.makedirs(isfigures)

        print('voxel_size:', voxel_size)

        for key, val in trees.items():

            inPR = (val) & (leaves) & (inds)
            pointsPR = POINTS[inPR]
            sensorsPR = SENSORS[inPR]

            m3att = lad.compute_attributes(pointsPR, resdir, voxel_size, key)

            results.append([downsample, voxel_size, (m3att == 1).sum(), (m3att == 2).sum(), (m3att == 3).sum()])

results = np.array(results)

total = np.array(np.sum(results[:,2:5], axis=1))
nI0 = results[:,2] / total
nP0 = results[:,3] / total
n00 = results[:,4] / total

nI = results[:,2]
nP = results[:,3]
n0 = results[:,4]


fig, (a0, a1) = plt.subplots(2, 1, gridspec_kw={'height_ratios': [2, 1]}, figsize=(10,22))
colors = plt.cm.jet(np.linspace(0,1,5))

for num, voxel_size in enumerate([0.1]):

    keep = (results[:,1] == voxel_size)
    downsample = results[:,0][keep]
    a0.plot(downsample*100, nP0[keep], marker='*', color=colors[num], label='%s' %(voxel_size))
    a0.plot(downsample*100, nI0[keep], marker='*', ls='--', color=colors[num])
    a0.plot(downsample*100, n00[keep], marker='*', ls=':', color=colors[num])
    
    a1.plot(downsample*100, nI[keep]/(nI[keep]+nP[keep]), marker='*', ls='-', color=colors[num])

text = '$n_{P}(k)$: Solid \n $n_{I}(k)$: Dashed \n $n_{0}(k)$: Dotted'
props = dict(boxstyle='round', facecolor='green', alpha=0.3)
a0.text(20, 0.5, text, fontsize=14, bbox=props)
a0.legend(title='Voxel Size')
a0.set_ylabel(r'$N$', size=20)

# a1.axhline(0.035, ls='--', c='k', lw=2)
a1.set_xlabel(r'Downsample (%)', size=20)
a1.set_ylabel(r'$n_{I}/(n_{I}+n_{P})$', size=20)
    


# Point Cloud Resolution (PCR)

In [ ]:
nbrs = NearestNeighbors(n_neighbors=2, algorithm='ball_tree').fit(POINTS[PCsample])
distances, indices = nbrs.kneighbors(POINTS[PCsample])

In [ ]:
dist = distances[:, 1]
dmin, dmax = np.percentile(dist, (0, 99.7))
keep = (dist >= dmin) & (dist <= dmax)
dist = dist[keep]
mean = np.round(np.mean(dist), 3)
median = np.round(np.median(dist), 3)
print(dmin, dmax)

plt.figure(figsize=(10, 6))

bins = np.linspace(dmin, dmax, 80)
plt.hist(dist, bins, density=True)
plt.axvline(mean, ls='--', c='k', label='Mean = %.3f' %(mean))
plt.axvline(median, ls='--', c='g', label='Median = %.3f' %(median))
plt.xlabel('Distance (m)', size=15)

plt.legend()
plt.show()

# Finding the correct voxel size

In [ ]:
def isinds(resdir, downsample):

    outdir = os.path.join(resdir, 'inds.npy')
    if os.path.exists(outdir):
        print('inds file already exists for donwnsample of %.3f at %s' %(downsample, outdir))

        inds = np.load(outdir)

        points = POINTS[inds]
        sensors = SENSORS[inds]

    else:

        print('inds not been created yet for donwnsample of %.3f' %(downsample))
        idx = np.random.randint(0, len(df), int(len(df) * downsample))
        inds = np.zeros(len(df), dtype=bool)
        inds[idx] = True

        points = POINTS[inds]
        sensors = SENSORS[inds]

        np.save(outdir, inds)

    return inds

def restmp(downsample, voxel_size, show=True):

    resdir = os.path.join(_data, name, 'lad_%s' %(str(downsample)))
    if not os.path.exists(resdir):
        os.makedirs(resdir)
    inds = isinds(resdir, downsample)

    for key, val in trees.items():

        inPR = (val) & (leaves) & (inds)
        pointsPR = POINTS[inPR]
        sensorsPR = SENSORS[inPR]

        density_overall, density, counts, m3scount = lad.density_counts(pointsPR, voxel_size)
        
        # dens_ratio = density / density_overall
        # print('Mean density ration: %.2f' %(np.mean(dens_ratio)))
        # print('Median density ration: %.2f' %(np.median(dens_ratio)))

        if show:

            fig = plt.figure(figsize=(10,6))
            plt.hist(counts, 25, align='left')
            plt.axvline(np.mean(counts), color='k', label='Mean counts = %.2f' %(np.mean(counts)))
            plt.xlabel(r'$n_{i}$', size=20)
            plt.legend()
            plt.show()

    return np.mean(counts)


def ds2vs_first(DS, PCR):

    N = np.cbrt(1/DS)
    VS = N * PCR
    
    return VS

def ds2vs(downsample, points, mean_counts=0, step=0.01, bounds=(1.25, 1.3)):

    # nbrs = NearestNeighbors(n_neighbors=2, algorithm='ball_tree').fit(points)
    # distances, indices = nbrs.kneighbors(points)

    minb, maxb = bounds

    # dist = distances[:, 1]
    # dmin, dmax = np.percentile(dist, (0, 99.7))
    # keep = (dist >= dmin) & (dist <= dmax)
    # dist = dist[keep]
    # PCR = np.round(np.mean(dist), 4)
    # print('PCR first guess: %.3f' %(PCR))
    PCR = 0.01

    # first guess for voxel size
    voxel_size = ds2vs_first(downsample, PCR)

    resdir = os.path.join(_data, name, 'lad_%s' %(str(downsample)))
    if not os.path.exists(resdir):
        os.makedirs(resdir)
    inds = isinds(resdir, downsample)

    for key, val in trees.items():

        inPR = (val) & (leaves) & (inds)
        pointsPR = POINTS[inPR]

        j = 0

        while (mean_counts < minb) or (mean_counts > maxb):

            # mean_counts = restmp(downsample, voxel_size, show=False)
            density_overall, density, counts, m3scount = lad.density_counts(pointsPR, voxel_size)
            mean_counts = np.mean(counts)
            
            print(mean_counts, voxel_size, step)

            if mean_counts < minb:
                voxel_size += step

            elif mean_counts > maxb:
                voxel_size -= step

            if np.abs(mean_counts - (maxb + minb)/2) < 0.1:
                step /= 2

    return voxel_size


In [ ]:
inPR = (trees['tree_0']) & (leaves)
voxels = {}

for DS in [0.01, 0.1]:

    VS = ds2vs(DS, POINTS[inPR], mean_counts=0, step=0.02, bounds=(1.4, 1.45))
    VS = np.round(VS, 3)
    voxels[DS] = VS

In [ ]:

plt.figure(figsize=(8,5))
plt.plot(100*np.array(list(voxels.keys())), np.array(list(voxels.values())), marker='o')
plt.xlabel('Downsampling (%)', size=18)
plt.ylabel('Voxel Size', size=18)

In [ ]:
for DS in [0.01, 0.1]:

    VS = voxels[DS]
    print(DS, VS)

    runall(downsample=DS, voxel_size=VS, kbins=1)

# Compute LAD

In [ ]:
restmp(downsample=0.05, voxel_size=0.051, show=True)

In [ ]:
for DS in [0.03, 0.05, 0.1]:

    VS = voxels[DS]
    print(DS, VS)
    get_rays(DS, VS)

In [ ]:
for DS in [0.01, 0.05]:

    VS = voxels[DS]
    print(DS, VS)

    runall(downsample=DS, voxel_size=VS, kbins=1)

# DTM

In [ ]:
POINTS = loads.DF2array(df[['x', 'y', 'z']])
# SENSORS = loads.DF2array(df[['xs', 'ys', 'zs']])

In [ ]:
keep = PCsample & (las.z < 0.8)
plt.figure(figsize=(16, 8))
plt.scatter(np.array(las.x)[keep][::1], np.array(las.z)[keep][::1], s=0.04, c='k')
plt.axhline(0.2, lw=1, c='k')

# res, bcmask = loads.remove_outliers(np.array(las.y)[keep], np.array(las.z)[keep], nbins=100, bounds=(0, 90))
# below[keep] |= ~bcmask

# plt.ylim(-1,4)

plt.xlabel(r'$x$', size=20)
plt.ylabel(r'$z$', size=20)

In [ ]:
keep = PCsample & (las.z < 0)
Ntot = keep.sum()

db = DBSCAN(eps=0.3).fit(points[keep])
# db = KMeans(n_clusters = 8, init='k-means++').fit(points[keep])

labels = {}

for i in set(db.labels_):
    mask = db.labels_ == i
    perc = 100*mask.sum()/Ntot
    if perc > 0:
        print(i, perc)
        labels[i] = mask

N = len(list(labels.keys()))

# plot the two point clouds
coltmp = plt.cm.jet(np.linspace(0,1,N))[:,0:3]

pointslist = [points[keep][val] for val in labels.values()]
colours = [list(i) for i in coltmp]
loads.showPCDS(pointslist, colours)

In [ ]:
alpha = 0.8
keep = PCsample & (las.z < 0)
pcd = loads.points2pcd(POINTS[keep], colors=[1,0,0])
downpcd = pcd.voxel_down_sample(voxel_size=0.05)
print(f"alpha={alpha:.3f}")
mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_alpha_shape(downpcd, alpha)
mesh.compute_vertex_normals()
# pcd = loads.points2pcd(POINTS[keep], colors=[1,0,0])
# downpcd = pcd.voxel_down_sample(voxel_size=0.05)
o3d.visualization.draw_geometries([downpcd, mesh.paint_uniform_color([0.5, 0.3, 0.1])], mesh_show_back_face=True)

In [ ]:
print('filter with average with 1 iteration')
mesh_out = mesh.filter_smooth_simple(number_of_iterations=2)
mesh_out.compute_vertex_normals()
print(mesh)
vertices = np.asarray(mesh_out.vertices)
pcd = loads.points2pcd(vertices, colors=[1,0,0])
mesh_name = os.path.join(_data, name, 'dtm_mesh.obj')
o3d.io.write_triangle_mesh(mesh_name, mesh_out)
o3d.visualization.draw_geometries([pcd, mesh_out], mesh_show_back_face=True)

In [ ]:
np.savetxt(os.path.join(_data,'DTM_vertices.csv'), vertices, fmt="%.8f", delimiter=",")

# To-Do

1) Fix algorithm to find `voxel size` from `downsampling`. It currently seems to work for the toy kiwifruit tree it isn't for the real data. Try:
   * Check that the algorithm realy work by computing several `LAD`s for different `downsampling` values using the toy model.
   * Remove outliers of foliage point cloud `FPC` as these might be adding noise to the mean of points per voxel. Try cliping or with a built-in `Open3D` function.
2) Improve `ray tracing` algorithm:
   * find a way to reduce the main sample i.e. `POINTS` and `SENSORS` to keep only the rays that pass trhough the plant region `PR`. It might be not that easy as many other process are involved.
3) Once `LAD` algorithm works. Usea the leaf inclination angles (`LIA`) and the leaf area index (`LAI`) as input parameters to built a realistic model of the kiwifruit trees. One way to do this is using `Blensor` software. Here, we can create a Python script to create the kiwifruit trees with the option of change some of the tree features such as the leaf sizes and leaf angles. Then, in a recursive algorithm, modify these values until we get the same `LIA` and `LAI` distributions from the real LiDAR.

# Dev Zone

In [ ]:
tot = len(df)

for DS in [0.005, 0.01, 0.05, 0.1, 0.2, 0.4]:

    idx = np.random.randint(0, len(df), int(len(df) * DS))

    print(DS, len(idx))
